In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## CFM 101: Group Assignment - Python Roboadvisor
### Team Number: 15
### Team Member Names: Landon Trinh, Ethan Zemelman, Jessie Deng
### Team Strategy Chosen: SAFE

## Goal
- Our team has decided to target dynamically building the safest portfolio
- Given a file of unknown stock tickers, our roboadvisor will run from November 25, 2023 to December 4, 2023
- The deviation will be calculated by taking calculating the difference between the final value of the portfolio and the initial portfolio value ($750,000)
- Ultimately, our aim is to have our portfolio deviate in nominal value as close to zero


## Introduction
> "Theory will only take you so far." - J. Robert Oppenheimer

In theory, our trading strategy should produce a portfolio that deviates in nominal value the least by taking into consideration the following:
- Beta
- Diversification
- Correlation
- Standard deviation
- Expected returns

Our goal will be to tell a convincing story to WHY we are picking our stocks. We will calculate and discuss statistics, display and intepret graphs, and explain our thought process

## 1. Setup
Before implementing our trading strategy, we will initialize required and useful constants as part of the rules:
- Currency of valid stocks (USD or CAD)
- Required average monthly volume (150,00 shares)
- The number of stocks we wish to purchase on the start date (10-22 stocks)
- Time interval (Janurary 1, 2023 - October 31, 2023)
- Minimum number of trading days for month (18 days)
- Minimum stock weighting: (100/2n)%, n = number of stocks in portfolio
- Maximum stock weighting (20%)
- Initial investment amount ($750,000 CAD)
- Buying date of roboadvisor (November 25, 2023 - December 4, 2023)
- Trading fee for each stock trade ($4.95 CAD)

In the end, our roboadvisor should create two DataFrames:
1. "Portfolio_Final"
- Index: Starts at 1 and ends at number of stocks in portfolio
- Headings: Ticker, Price (price of stock on Nov 25), Currency (CAD or USD), Shares, Value, Weight (adds to 100%)

2. "Stocks_Final"
We should output this DataFrame to a CSV file titled "Stocks_Group_15.csv"
- Index: Same as "Portfolio Final"
- Headings: Tickers and Shares from "Portfolio_Final"


In [2]:
# Investment amount (CAD)
capital = 750000

# Number of stocks to buy for portfolio
num_stocks = 15

# Maximum and minimum weightings of each stock in portfolio
min_weight = 1 / (2 * num_stocks)
max_weight = 0.20

# Start and end date for roboadvisor
# start_date = "2023-11-25"
# end_date = "2023-12-04"

# Filtering requirements
valid_currency = ["CAD", "USD"]
min_trading_days = 18
required_avg_volume = 150000

## 2. Filtering
After reading in the CSV file containg stock tickers, we must filter the list of stocks to make sure they are valid stock tickers according to the following rules:
- Include stocks that have an average monthly volume of at leaest 150,000 shares based on Jan 1, 2023 - Oct 31, 2023 (drop any months that don't have at least 18 trading days)
- Stock denominated in USD or CAD

In [3]:
# Read in CSV ticker file
tickers = pd.read_csv("tickers_example.csv", header=None)
tickers = tickers.rename(columns={0: "ticker"})
tickers_lst = tickers["ticker"].tolist()
tickers_lst

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'AGN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CELG',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MON',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTN',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

In [4]:
# Set parameters
filter_start_date = "2023-01-01" # Jan 1, 2023
filter_end_date = "2023-10-31" # Oct 31, 2023
filter_interval = "1mo"

In [5]:
# Keep stocks with average monthly volume of 150k shares - drop months with less than 18 trading days
def valid_volume(stock_ticker):

    monthly_volumes = []

    for month in range(1,10):
        # Set monthly date intervals
        month_start_date = str(date(2023, month, 1))
        month_end_date = str(date(2023, month + 1, 1))

        # Retrieve volume data for month
        monthly_volume_hist = stock_ticker.history(interval="1d", start=month_start_date, end=month_end_date).Volume
        monthly_volume_hist.index = monthly_volume_hist.index.strftime("%Y-%m-%d")

        # Retrive number of trading days and averge monthly volume
        trading_days = len(monthly_volume_hist)
        monthly_volume = monthly_volume_hist.sum()

        # Add monthly volume to list if valid number of trading days
        if trading_days >= min_trading_days:
            monthly_volumes.append(monthly_volume)

    # Determine whether average monthly volume meets requirement
    avg_monthly_volume = sum(monthly_volumes) / len(monthly_volumes)
    if avg_monthly_volume < required_avg_volume:
        return False
    return True

In [6]:
# Retrieve stock data into DataFrame
def get_stock_data(tickers):
    
    stock_data = pd.DataFrame()
    
    for ticker in tickers:
        try:
            # Get base currency
            stock_ticker = yf.Ticker(ticker)
            base_currency = stock_ticker.fast_info["currency"]
            
            # Retrieve and store CAD/USD stock data in DataFrame
            if base_currency in valid_currency and valid_volume(stock_ticker):
                stock_price_hist = stock_ticker.history(interval=filter_interval, start=filter_start_date, end=filter_end_date).Close
                stock_price_hist.index = stock_price_hist.index.strftime("%Y-%m-%d")
                stock_data[ticker] = stock_price_hist
        except:
            print(f"{ticker} may be delisted")
        
    return stock_data.dropna()

get_stock_data(tickers_lst)

AGN may be delisted
CELG may be delisted
MON may be delisted
RTN may be delisted


,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,143.487961,142.012970,108.381836,274.685822,62.022976,103.129997,172.380508,213.000000,34.683270,290.899994,...,130.563614,130.533325,65.570000,27.465641,88.929955,171.818436,493.463745,200.284302,178.145782,47.695686
2023-02-01,146.590622,149.330322,100.176537,262.466217,59.952927,94.230003,172.048508,201.550003,33.529766,269.859985,...,121.075920,134.125229,56.180000,25.990332,88.720131,167.418335,470.480469,203.315201,175.510590,45.713158
2023-03-01,164.233795,154.637909,99.723518,282.490906,49.406471,103.290001,163.109375,212.429993,27.957766,278.029999,...,125.045464,125.131279,64.800003,25.702934,79.052216,181.635941,467.168884,198.741989,188.214035,34.526699
2023-04-01,168.994461,146.632874,108.793770,277.035034,52.385254,105.449997,159.539673,206.779999,28.808136,304.230011,...,115.178925,130.223663,65.639999,27.871626,80.145966,163.268265,488.177795,193.251556,174.456238,33.272427
2023-05-01,176.533890,135.095108,100.954300,303.554749,52.177849,120.580002,157.371536,205.699997,27.342150,296.410004,...,111.835968,118.718704,77.669998,24.960245,75.933998,169.791183,483.366333,190.111343,162.027649,29.021166
2023-06-01,193.453568,131.932159,107.902328,306.194183,56.829704,130.360001,172.894302,211.160004,28.227644,284.850006,...,118.203018,123.695053,85.620003,25.018475,81.141869,177.140182,476.818848,203.429489,175.583862,32.068874
2023-07-01,195.926956,146.473801,110.188652,313.904114,59.922195,133.679993,167.614197,238.850006,31.735537,270.190002,...,131.240707,127.811043,89.080002,23.112240,85.934677,177.120514,504.406372,230.670074,183.302704,39.094318
2023-08-01,187.369797,145.503586,102.333519,322.432709,58.172646,138.009995,157.343521,224.029999,28.433058,267.359985,...,113.724731,120.251060,89.889999,23.358324,82.419998,166.533463,474.731903,219.286697,165.935303,35.990810
2023-09-01,170.984741,147.582764,96.316818,305.841522,60.240295,127.120003,148.579529,191.679993,27.379999,257.010010,...,111.059998,117.248245,74.139999,21.832602,81.830002,157.565659,502.234833,203.630005,154.065704,32.572033


## 3. Stock Analysis
- Get standard deviation of each stock
- Get the beta of each stock
- Get the expected returns of each stock
- Create a correlation matrix of all the stocks

## 4. Portfolio Optimization
- Create random weights for each stock, and create n number of random portfolios
- Choose the portfolio with the lowest expected returns

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.